In [2]:
import spreg
from spreg import OLS
import pysal as ps
import libpysal
from pysal.lib import weights
import geopandas as gpd
import pandas as pd
import numpy as np

/home/maciek/.pyenv/versions/3.8.12/envs/school-map-project/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
gdf = gpd.read_file("../raw_data/full.shp")

In [ ]:
gdf.geometry

In [2]:
df = pd.read_csv('../raw_data/data_dropped.csv')

In [4]:
qw = libpysal.weights.Queen.from_file("../raw_data/more_maps/schools.shp")

In [5]:
qw

[(388591.4, 5820578.3),
 (387241.9, 5820470.8),
 (390195.6, 5824246.9),
 (388376.7, 5824343.1),
 (391299.1, 5819271.1),
 (391497.3, 5821994.0),
 (390876.3, 5821514.2),
 (391579.6, 5820818.5),
 (390139.2, 5820929.7),
 (393086.4, 5819617.3),
 (390255.0, 5819340.8),
 (391988.2, 5818679.4),
 (388286.2, 5821296.9),
 (388450.2, 5819936.9),
 (388517.0, 5820392.4),
 (386866.8, 5821160.2),
 (387392.5, 5820330.0),
 (388622.5, 5824180.5),
 (391221.5, 5823605.1),
 (390356.2, 5823816.8),
 (388633.0, 5822511.0),
 (390415.2, 5824286.9),
 (389045.9, 5823135.1),
 (390376.2, 5824682.2),
 (390157.6, 5822723.0),
 (390796.3, 5825153.9),
 (391031.7, 5823114.6),
 (390917.4, 5822089.8),
 (391284.1, 5822538.3),
 (387112.3, 5824047.1),
 (388694.4, 5823529.9),
 (387578.1, 5823564.6),
 (389725.7, 5823382.0),
 (388885.1, 5818262.5),
 (388918.1, 5822867.2),
 (391293.4, 5820954.5),
 (386800.8, 5820560.5),
 (389631.7, 5822143.0),
 (390553.5, 5823573.8),
 (387978.6, 5823131.3),
 (391041.9, 5822217.6),
 (385794.1, 5820

In [3]:
df.to_numpy().shape

(542, 43)

In [4]:
df.drop(columns=[
    'ant_arbeit', 'ant_transf', 'ant_arbe_1', 'ant_tran_1', 'Kinderar_1',
    'PLR_ID', 'MH_E', 'geometry', 'aenderung_'
],
        inplace=True)

In [ ]:
gdf.drop(columns=[
    'ant_arbeit', 'ant_transf', 'ant_arbe_1', 'ant_tran_1', 'Kinderar_1',
    'PLR_ID', 'MH_E', 'geometry', 'aenderung_'
],
        inplace=True)

In [6]:
y = np.array(df['Kinderarmu'])
X = df.drop(columns=['Kinderarmu']).to_numpy()

In [9]:
basic_ols = OLS(y, X, name_y='Kinderarmut', name_ds='basic_model', nonspat_diag=True)

In [10]:
print(basic_ols.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            : basic_model
Weights matrix      :        None
Dependent Variable  : Kinderarmut                Number of Observations:         542
Mean dependent var  :     24.5406                Number of Variables   :          34
S.D. dependent var  :     16.5774                Degrees of Freedom    :         508
R-squared           :      0.7359
Adjusted R-squared  :      0.7187
Sum squared residual:   39270.854                F-statistic           :     42.8850
Sigma-square        :      77.305                Prob(F-statistic)     :  7.341e-125
S.E. of regression  :       8.792                Log likelihood        :   -1929.750
Sigma-square ML     :      72.455                Akaike info criterion :    3927.500
S.E of regression ML:      8.5121                Schwarz criterion     :    4073.539

-----------------------------------------------------------------------------